# Invitations Alter Verwalter

In [1]:
# imports
import pandas as pd
import numpy as np
import datetime as dt  # dealing with dates
from random import randrange  # returns a randomly selected element from the specified range
from fpdf import FPDF  # creating pdf in python
from pdflatex import PDFLaTeX  # creating pdf from latex template
import barcode as bc  # generate barcodes
from pathlib import Path  # filesystem paths with appropriate semantics
import os  # interacting with filesystem
from docx2pdf import convert
from docx import Document
from PyPDF2 import PdfMerger
import os

In [2]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(0, int_delta, 240)  # start stop step
    return start + dt.timedelta(seconds=random_second)

In [3]:
def generate_barcode(string, name_dir, save_as="barcode"):
    """
    takes input string and encodes it as barcode which is saved to the current directroy
    """
    my_code = bc.codex.Code128(str(string), writer=bc.writer.ImageWriter())
    my_code.default_writer_options['write_text'] = False
    my_code.save(f"{name_dir}/{save_as}")  # saves as png because fpdf cant handle svg

In [4]:
def generate_ticket(name, city, time_slot, name_dir, save_as="ticket.pdf"):
    """
    create entry ticket with barcode for input person and timeslot and saves it to given directory
    """
    # create pdf object
    pdf = FPDF('P', 'mm', 'A4')
    pdf.add_page()

    # insert barcode
    pdf.image(f"{name_dir}/barcode.png", 105, 28, 95)

    ticket_text = "Dieses Ticket berechtigt die unten genannte Person zu Teilnahme an der unten genannten Veranstaltung. Ticket im unten indizierten Zeitrahmen unaufgefordert am Einlass vorzeigen."

    # create table
    time_slot_end = time_slot + dt.timedelta(seconds=240)
    table_data = (
        ("Veranstaltung", "Alter Verwalter!"),
        ("Gast",  name),
        ("Gültigkeitszeitraum", str(time_slot) + " - " + str(time_slot_end.time())),
        ("Veranstatungsort", "Mommsenstraße 1, 01069 Dresden")
    )

    # insert headline#
    pdf.set_font('courier', '', 20)
    pdf.cell(50, 10, txt="Ticket: Alter Verwalter!", fill=False, ln=True)
    pdf.set_font('courier', '', 12)
    pdf.set_text_color(0, 0, 0)
    pdf.cell(10, 10, ln=True)
    pdf.multi_cell(90, 5, txt=ticket_text, border=False, ln=False)
    pdf.cell(10, 10, ln=True)

    # define table
    line_height = pdf.font_size * 2.5
    col_width = pdf.epw / 2  # distribute content evenly
    for row in table_data:
        for datum in row:
            pdf.multi_cell(col_width, line_height, datum, border=1,
                    new_x="RIGHT", new_y="TOP", max_line_height=pdf.font_size)
        pdf.ln(line_height)

    pdf.output(f"{name_dir}/ticket_{name.replace(' ', '_')}_{city}.pdf")

In [5]:
def update_invitation_template(name, surname, gender, adress, template, name_dir):
    """
    updates latex template and inserts name and adress
    """
    
    # access file and replace the placeholders in the template
    if gender == "m":
        salutation = "geehrter Herr"
    elif gender == "w":
        salutation = "geehrte Frau"
    else:
        print("ungültiges Geschlecht")
        sys.exit()  
    
    # open template
    with open(template, 'r', encoding='utf8') as f:
        file = f.read() \
        .replace("<anrede>", salutation) \
        .replace("<name>", str(name)) \
        .replace("<adress>", adress) \
        .replace("<surname>", surname) \
        .replace('ü', '{\"u}').replace('ä', '{\"a}').replace('ö', '{\"o}').replace('ß', '{\"s}')

    # create updated template
    with open(f"{name_dir}/invitation_{name.replace(' ', '_')}.tex", 'w') as f2:
        f2.write(file)
        f2.close()

In [6]:
def generate_invitation(name, city, template, name_dir):
    """
    creates invitation from template
    """
    pdfl = PDFLaTeX.from_texfile(f'{name_dir}/{template}')
    pdf, log, completed_process = pdfl.create_pdf(keep_pdf_file=True, keep_log_file=True)
    os.replace(f"invitation_{name.replace(' ', '_')}.log" ,f"{name_dir}/invitation_{name.replace(' ', '_')}_{city}.log")
    os.replace(f"invitation_{name.replace(' ', '_')}.pdf" ,f"{name_dir}/invitation_{name.replace(' ', '_')}_{city}.pdf")

In [7]:
def update_absence(name, adress, template_tex, template_lco, name_dir):
    """
    updating abscence templates
    """

    # update template
    with open(template_tex, 'r', encoding="utf-8") as f:
        file = f.read() \
        .replace("<name>", name) \
        .replace("<adress>", adress) \
        .replace('ü', '{\"u}').replace('ä', '{\"a}').replace('ö', '{\"o}').replace('ß', '{\"s}') \
        .replace("<name_underscore>", name.replace(" ", "_"))
        
    # create updated template
    with open(f"{name_dir}/absence_{name.replace(' ', '_')}.tex", 'w', encoding='utf-8') as f2:
        f2.write(file)
        f2.close()
        
    # open .lco-file
    with open(template_lco, 'r', encoding='utf-8') as f:
        file = f.read() \
        .replace("<name>", name) \
        .replace("<adress>", adress)
    
    # create updated .lco file
    with open(f"{name_dir}/{template_lco}", 'w', encoding='utf-8') as f2:
        f2.write(file)
        f2.close()

In [8]:
def generate_absence(name, city, template, name_dir):
    """
    creates invitation from template
    """
    pdfl = PDFLaTeX.from_texfile(f'{name_dir}/{template}')
    pdf, log, completed_process = pdfl.create_pdf(keep_pdf_file=True, keep_log_file=True)
    os.replace(f"absence_{name.replace(' ', '_')}.log" ,f"{name_dir}/absence_{name.replace(' ', '_')}_{city}.log")
    os.replace(f"absence_{name.replace(' ', '_')}.pdf" ,f"{name_dir}/absence_{name.replace(' ', '_')}_{city}.pdf")

In [9]:
def merge_pdfs(print_location, search_dir):
    
    # get list of all pdfs for certain city    
    pdfs = list()
    
    for path, subdirs, files in os.walk(search_dir):
        for name in files:
            if name.endswith(f'{print_location}.pdf'):
                pdfs.append(os.path.join(path, name))
    # merge pdfs
    merger = PdfMerger()

    for pdf in pdfs:
        merger.append(pdf)

    merger.write(f"result_{print_location}.pdf")
    merger.close()

### Create dataframe from .csv file

In [10]:
# read from local .csv-file
df = pd.read_csv('teilnahmeliste.csv')
# dataframe vervollständigen
d1 = dt.datetime.strptime('28/1/2023 7:00 PM', '%d/%m/%Y %I:%M %p')
d2 = dt.datetime.strptime('28/1/2023 8:00 PM', '%d/%m/%Y %I:%M %p')
df["Name"]=df["Vorname"] + " " + df["Nachname"]
df["Zeitslot"]=df.apply(lambda row: random_date(d1, d2), axis=1)  # give everyone their timeslot
df.head()

,Vorname,Nachname,Adresse,Geschlecht,Wo drucken?,Zeitslot,Name
0,Freida,Nurgül,"Leopoldstraße 37, 12161 Berlin",w,Dresden,2023-01-28 19:16:00,Freida Nurgül
1,Gladys,Ronan,"Sömmeringstr. 95, 89260 Weißenhorn",m,Dresden,2023-01-28 19:28:00,Gladys Ronan
2,Anil,Ina,"Flughafenstrasse 94, 92526 Oberviechtach",w,Leipzig,2023-01-28 19:08:00,Anil Ina
3,Hjörtur,Sargis,"C/ Eras 60, 2D, 28722 El Vellón, Spain",m,Dresden,2023-01-28 19:12:00,Hjörtur Sargis
4,Lungile,Haig,"14 boulevard Albin Durand, 95000 Cergy, France",w,Marseille,2023-01-28 19:04:00,Lungile Haig


### Create folder hierarchy and call the functions for each guest

In [11]:
# create subfolder for every entry
# TODO: create barcode only if not already existing?; check if it is consistent with timeslot (aka control)
for i in df.index:
    
    # set all variables
    # read from dataframe
    name = df["Name"][i]
    surname = df["Nachname"][i]
    time_slot = df["Zeitslot"][i]
    gender = df["Geschlecht"][i]
    adress = df["Adresse"][i].replace(",", "\\\\")  # lololololol! to recieve \\ we have to escape it
    city = df["Wo drucken?"][i]
    # to create dirs
    folder_name = (df["Name"][i]).replace(" ", "_")
    name_dir = f"./einladungen/{folder_name}"
    # to update ticket
    tex_template = "brief.tex"
    # to update absence
    absence_template = "absage.tex"
    lco_template = "sender2.lco"
    
    # create directories
    Path(f"./einladungen/{folder_name}").mkdir(parents=True, exist_ok=True)  # create subfolder if not existing
    nmae_dir = f"./einladungen/{folder_name}"
    
    # create barcodecode
    generate_barcode(time_slot, name_dir)
    
    # create ticket
    generate_ticket(name, city, time_slot, name_dir)
    
    # update .tex template and write to local folder
    update_invitation_template(name, surname, gender, adress, tex_template, name_dir)
    
    # generate invitation
    invitation_updated = f"invitation_{(df['Name'][i]).replace(' ', '_')}.tex"
    generate_invitation(name, city, invitation_updated, name_dir)
    
    # update absnece tex and lco files
    update_absence(name, adress, absence_template, lco_template, name_dir)
    
    # generate absence
    absence_updated = f"absence_{(df['Name'][i]).replace(' ', '_')}.tex"
    generate_absence(name, city, absence_updated, name_dir)
    

# concatenate pdfs
for city in ["Dresden", "Leipzig", "Marseille"]:
    merge_pdfs(city, ".")

C:\Users\Padman\AppData\Local\Temp\ipykernel_10424\2288567491.py:25: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(50, 10, txt="Ticket: Alter Verwalter!", fill=False, ln=True)
C:\Users\Padman\AppData\Local\Temp\ipykernel_10424\2288567491.py:28: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(10, 10, ln=True)
C:\Users\Padman\AppData\Local\Temp\ipykernel_10424\2288567491.py:29: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=False use new_x=XPos.RIGHT, new_y=YPos.NEXT.
  pdf.multi_cell(90, 5, txt=ticket_text, border=False, ln=False)
C:\Users\Padman\AppData\Local\Temp\ipykernel_10424\2288567491.py:30: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(10, 10, ln=True)


In [12]:
print("you did it!")

you did it!
